In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
tf.__version__

'2.10.0'

In [2]:
data=tf.keras.preprocessing.image_dataset_from_directory("C:\\Users\\ALVI COMPUTER\\Desktop\\ML1_dataset",
                                                         shuffle=True,
                                                         batch_size=32
                                                        )

Found 862 files belonging to 4 classes.


In [3]:
data.class_names

['Apple', 'Banana', 'Mangoe', 'Orange']

In [4]:
len(data)

27

In [5]:
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
from PIL import Image

In [7]:
def partition(data,shuffle=True,shuffle_size=10000):
    if shuffle:
        data=data.shuffle(shuffle_size,seed=12)
        train_size=0.8*len(data)
        train_data=data.take(int(train_size))
        rem_data=data.skip(int(train_size))
        test_size=len(data)*0.1
        test_data=rem_data.take(int(test_size))
        val_data=rem_data.skip(int(test_size))
        return train_data,test_data,val_data

In [8]:
train_data,test_data,val_data=partition(data)

In [9]:
train_data=train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data=test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_data=val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [10]:
from tensorflow.keras import layers,models
resize_rescale_layer=tf.keras.Sequential(layers.experimental.preprocessing.Resizing(256,256),
                                          layers.experimental.preprocessing.Rescaling(1.0/255))

In [11]:
data_augmentation_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2) 
])

In [12]:
channels=3
batch_size=32
image_size=256

In [21]:
input_shape=(image_size,image_size,channels)
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255),
    data_augmentation_layer,
    layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [23]:
model.fit(train_data,
          epochs=30,
          batch_size=32,
          verbose=1,
          validation_data=val_data)

Epoch 1/30
21/21 [==============================] - 63s 2s/step - loss: 1.0671 - accuracy: 0.5209 - val_loss: 0.6771 - val_accuracy: 0.7266
Epoch 2/30
21/21 [==============================] - 49s 2s/step - loss: 0.5647 - accuracy: 0.8164 - val_loss: 0.3839 - val_accuracy: 0.9219
Epoch 3/30
21/21 [==============================] - 50s 2s/step - loss: 0.4403 - accuracy: 0.8612 - val_loss: 0.3204 - val_accuracy: 0.9375
Epoch 4/30
21/21 [==============================] - 50s 2s/step - loss: 0.4851 - accuracy: 0.8134 - val_loss: 0.4900 - val_accuracy: 0.7969
Epoch 5/30
21/21 [==============================] - 49s 2s/step - loss: 0.4492 - accuracy: 0.8522 - val_loss: 0.3394 - val_accuracy: 0.9062
Epoch 6/30
21/21 [==============================] - 49s 2s/step - loss: 0.3165 - accuracy: 0.9015 - val_loss: 0.3050 - val_accuracy: 0.9297
Epoch 7/30
21/21 [==============================] - 50s 2s/step - loss: 0.3428 - accuracy: 0.8746 - val_loss: 0.2095 - val_accuracy: 0.9609
Epoch 8/30
21/21 [==

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 256, 256, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 64)      18496     
                                                      

In [25]:
model.evaluate(test_data)

2/2 [==============================] - 1s 455ms/step - loss: 0.0264 - accuracy: 0.9844


[0.026445861905813217, 0.984375]

In [26]:
model.save('my_model2.h5')

In [27]:
data_fahad=tf.keras.preprocessing.image_dataset_from_directory("C:\\Users\\ALVI COMPUTER\\Downloads\\test_data_cnn",
                                                         shuffle=True,
                                                         batch_size=10
                                                        )

Found 9 files belonging to 4 classes.


In [28]:
model.evaluate(data_fahad)

1/1 [==============================] - 0s 205ms/step - loss: 2.3213 - accuracy: 0.6667


[2.32127046585083, 0.6666666865348816]

In [ ]:
import os
clas=['Apple', 'Banana', 'Mangoe', 'Orange']
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    return image

def predict_images_in_folder(folder_path, model):
    images = []
    image_paths = []

    # Iterate through the folder and load images
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            image = load_and_preprocess_image(image_path)
            images.append(image)
            image_paths.append(image_path)

    # Convert list of images to a TensorFlow batch
    images = tf.stack(images, axis=0)

    # Predict using the model
    predictions = model.predict(images)

    # Print or return predictions
    for i, prediction in enumerate(predictions):
        print(f"Prediction for {image_paths[i]}: {np.argmax(prediction)}")
        plt.imshow(images[i])
        plt.title(f"Prediction: {clas[np.argmax(predictions)]}")
        plt.axis('off')
        plt.show()

# Example usage
folder_path = 'C:\\Users\\ALVI COMPUTER\\Downloads\\test_data_single_folder'

predict_images_in_folder(folder_path, model)


1/1 [==============================] - 0s 248ms/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Prediction for C:\Users\ALVI COMPUTER\Downloads\test_data_single_folder\apple.jpeg: 0


IndexError: list index out of range